In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)


test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}") #X are the images C corresponsin to the channels (here 1 this means greyscale)
    print(f"Shape of y: {y.shape} {y.dtype}") #y are the 64 labels
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
device = ( "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

# Define model, class inherits forom nn.Module
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() #initialize its superclass nn.Module
        self.flatten = nn.Flatten() #reshapes (1, 28, 28) to (784,)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad(): #disables gradient computation
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.286653  [   64/60000]
loss: 2.276192  [ 6464/60000]
loss: 2.252327  [12864/60000]
loss: 2.253839  [19264/60000]
loss: 2.236441  [25664/60000]
loss: 2.208475  [32064/60000]
loss: 2.220511  [38464/60000]
loss: 2.175694  [44864/60000]
loss: 2.164979  [51264/60000]
loss: 2.153354  [57664/60000]
Test Error: 
 Accuracy: 40.4%, Avg loss: 2.131536 

Epoch 2
-------------------------------
loss: 2.132707  [   64/60000]
loss: 2.123711  [ 6464/60000]
loss: 2.058680  [12864/60000]
loss: 2.085600  [19264/60000]
loss: 2.034278  [25664/60000]
loss: 1.972026  [32064/60000]
loss: 2.001269  [38464/60000]
loss: 1.907214  [44864/60000]
loss: 1.908077  [51264/60000]
loss: 1.858829  [57664/60000]
Test Error: 
 Accuracy: 59.9%, Avg loss: 1.840730 

Epoch 3
-------------------------------
loss: 1.863408  [   64/60000]
loss: 1.841869  [ 6464/60000]
loss: 1.712652  [12864/60000]
loss: 1.768564  [19264/60000]
loss: 1.662127  [25664/60000]
loss: 1.615287  [32064/600